In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_18028/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_18028/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_40_7,0.987662,0.724320,0.740785,0.990070,0.004645,0.163655,0.181586,0.004010,0.028836,0.068153,0.983420,0.068683,76.744023,171.372660,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_41_5,0.987640,0.723455,0.741047,0.982368,0.004653,0.164169,0.181402,0.007216,0.028755,0.068212,0.983392,0.068742,76.740563,171.369200,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_41_3,0.987673,0.723134,0.740837,0.982718,0.004640,0.164360,0.181549,0.007073,0.028755,0.068121,0.983436,0.068651,76.745904,171.374541,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_40_8,0.987717,0.723053,0.738714,0.989515,0.004624,0.164408,0.183036,0.004234,0.028840,0.067999,0.983495,0.068528,76.753054,171.381691,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_41_0,0.987702,0.723049,0.740007,0.983384,0.004629,0.164410,0.182130,0.006800,0.028766,0.068040,0.983475,0.068569,76.750654,171.379291,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,model_3_0_0,0.051128,-0.018215,-0.128465,0.013355,0.357201,0.604457,0.790513,0.312141,0.306513,0.597663,-0.530057,0.602314,100.058912,240.568100,"Hidden Size=[12], regularizer=0.02, learning_r..."
1496,model_2_0_0,0.166408,-0.046749,0.123356,0.097177,0.313804,0.621396,0.614107,0.401203,0.298857,0.560182,-0.120139,0.564542,68.317972,162.946609,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1497,model_2_0_3,0.199577,-0.063274,0.062105,0.159015,0.301318,0.631206,0.657015,0.373723,0.298196,0.548924,-0.075569,0.553196,68.399179,163.027816,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1498,model_1_0_0,-0.260867,-0.264620,-0.470295,-0.298213,0.474651,0.750733,1.029973,0.413742,0.363941,0.688949,-0.452249,0.694311,35.490350,84.238436,"Hidden Size=[4], regularizer=0.02, learning_ra..."
